In [1]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

epochs = 30
batch_size = 64

!rm -rf ./logs/hypothyroid/*

In [2]:
import numpy as np
from sklearn import preprocessing
from sklearn.datasets import fetch_openml


openml_dataset = fetch_openml(
    name="hypothyroid", version=4, as_frame=False, parser="liac-arff"
)

n_total, dims = openml_dataset.data.shape
n = int(n_total * 0.7)

X = openml_dataset.data.astype(np.float32)
y = preprocessing.LabelEncoder().fit_transform(openml_dataset.target)

ds_raw = tf.data.Dataset.from_tensor_slices((X, y)).shuffle(n_total)

ds_train_raw = ds_raw.take(n)
ds_test_raw = ds_raw.skip(n)

n_classes = np.unique(openml_dataset.target).shape[0]

print("n: ", n, "n_classes: ", n_classes, "dims: ", dims)


OpenMLError: Dataset hyperthyroid with version 4 not found.

In [ ]:
ds_train_normalized = ds_train_raw.cache()

ds_test_normalized = ds_test_raw.cache()


def prepare(ds, batch_size=batch_size):
    return ds.shuffle(n).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
RandomFourierFeatures = keras.layers.experimental.RandomFourierFeatures

model_svm = keras.Sequential(
    [
        layers.Input(shape=(dims,)),
        RandomFourierFeatures(
            output_dim=2000, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=n_classes),
    ]
)
model_svm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)

modeldir = "./logs/hypothyroid/linear-8192-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_svm.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(
            log_dir=modeldir + "/log",
            histogram_freq=1,
        )
    ],
)
model_svm.save(modeldir + "/model")


In [ ]:
def gen_samples(psi, t=1000):
    return [
        list(
            ds_train_raw.shuffle(n)
            .take(psi)
            .batch(psi)
            .as_numpy_iterator()
        )[0][0]
        for _ in range(t)
    ]


def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


In [ ]:
class IsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
        super(IsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p
        self.soft = soft

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p, self.soft)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
                "soft": self.soft,
            }
        )
        return config


def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(t_samples[i], p=p, soft=soft, name="ann_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [ ]:
def _tf_ann_weighted(X, samples, sample_weights, p=2, soft=True):
    m_dis = None  # [n, psi]
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]  # [i, dims]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )  # [n, 1]
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    m_dis = m_dis * sample_weights

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


class FlexibleIsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, **kwargs):
        super(FlexibleIsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.sample_weights = self.add_weight(
            name="dimential_weights",
            shape=(
                1,
                self.samples.shape[0],
            ),
            initializer="ones",
            trainable=True,
        )
        super(FlexibleIsolationEncodingLayer, self).build(input_shape)

    def call(self, inputs):
        return _tf_ann_weighted(inputs, self.samples, self.sample_weights, self.p)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
            }
        )
        return config


def build_flex_model(t_samples, p=2):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        FlexibleIsolationEncodingLayer(t_samples[i], p=p, name="ann_flex_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [ ]:
t_samples = gen_samples(psi=20, t=50)


model_hard_20_50 = build_model(t_samples, soft=False)
modeldir = "./logs/hypothyroid/hard-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_20_50.save(modeldir + "/model")


In [ ]:
model_soft_20_50 = build_model(t_samples, soft=True)
modeldir = "./logs/hypothyroid/soft-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_20_50.save(modeldir + "/model")


In [ ]:
model_flex_20_50 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/hypothyroid/flex-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_20_50.save(modeldir + "/model")


In [ ]:
t_samples = gen_samples(psi=100, t=10)

model_hard_100_10 = build_model(t_samples, soft=False)
modeldir = "./logs/hypothyroid/hard-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_100_10.save(modeldir + "/model")


In [ ]:
model_soft_100_10 = build_model(t_samples, soft=True)
modeldir = "./logs/hypothyroid/soft-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_100_10.save(modeldir + "/model")


In [ ]:
model_flex_100_10 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/hypothyroid/flex-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_100_10.save(modeldir + "/model")
